This notebook uses [mvncall](https://mathgen.stats.ox.ac.uk/genetics_software/mvncall/mvncall.html) to phase two multiallelic SNPs within VGSC and to add back in the insecticide resistance linked N1570Y SNP filtered out of the PASS callset.

In [1]:
%run setup.ipynb

## install mvncall
- mvncall depends on a couple of boost libraries, I installed these first using "sudo apt-get install libboost-dev"

In [1]:
%%bash --err install_err --out install_out

# This script downloads and installs mvncall. We won't include this in 
# the standard install.sh script as this is not something we want to do
# as part of continuous integration, it is only needed for this data 
# generation task.

set -xeo pipefail

cd ../dependencies

if [ ! -f mvncall.installed ]; then
    echo installing mvncall
    
    # clean up
    rm -rvf mvncall*
    
    # download and unpack
    wget https://mathgen.stats.ox.ac.uk/genetics_software/mvncall/mvncall_v1.0_x86_64_dynamic.tgz
    tar zxvf mvncall_v1.0_x86_64_dynamic.tgz
    
    # trick mnvcall into finding boost libraries - their names aren't what mvncall expects
    locate libboost_iostreams | xargs -I '{}' ln -v -f -s '{}' libboost_iostreams.so.5
    locate libboost_program_options | xargs -I '{}' ln -v -f -s '{}' libboost_program_options.so.5
    
    # try running mvncall
    export LD_LIBRARY_PATH=.
    ./mvncall_v1.0_x86_64_dynamic/mvncall 
    
    # mark success
    touch mvncall.installed

else
    echo mvncall already installed
    
fi

In [2]:
#check install
print(install_out)

mvncall already installed



In [3]:
# check we can run mvncall
mvncall = 'LD_LIBRARY_PATH=../dependencies ../dependencies/mvncall_v1.0_x86_64_dynamic/mvncall'
!{mvncall}



  MVNcall version 1.0


Copyright 2013 Androniki Menelaou and Jonathan Marchini
Please see the LICENCE file included with this program for conditions of use.


INPUT FILES

    --sample-file <file>                A list of the sample IDs names 
    --glfs <file>                       Genotype likelihoods input file (vcf format)
    --scaffold-file <file>              Haplotype scaffold 


MODEL OPTIONS

    --int <lower> <upper>               Lower and upper bounds (physical positions) of the region of interest. 
    --list <file>                       A list of all positions to be tested [instead of interval] 
    --numsnps <int>                     Number of SNPs at either site of the position of interest [default 50]
    --lambda <double>                   Penalty value [default 0.06]
    --ped-file <file>                   File in .ped format
    --k <int>                           The number of haplotypes to be selected for inference, [default k = all ]
    --model-avg          

## prepare input files

In [5]:
# these are the source data files for the phasing
sample_file = '../ngs.sanger.ac.uk/production/ag1000g/phase1/AR3.1/haplotypes/main/shapeit/ag1000g.phase1.ar3.1.haplotypes.2L.sample.gz'
vcf_file = '../ngs.sanger.ac.uk/production/ag1000g/phase1/AR3/variation/main/vcf/ag1000g.phase1.ar3.2L.vcf.gz'
scaffold_file = '../ngs.sanger.ac.uk/production/ag1000g/phase1/AR3.1/haplotypes/main/shapeit/ag1000g.phase1.ar3.1.haplotypes.2L.haps.gz'

## list of SNPs to phase

In [2]:
# this file will contain the list of SNPs to be phased
list_file = '../data/phasing_extra_phase1.list'

In [3]:
%%file {list_file}
2391228
2400071
2429745

Writing ../data/data_phasing_phase1.list


In [8]:
# for mvncall we need a simple manifest of sample IDs
# N.B., we will exclude the cross parents
!gunzip -v {sample_file} -c | head -n 767 | tail -n 765 | cut -d' ' -f1 > /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.sample

../ngs.sanger.ac.uk/production/ag1000g/phase1/AR3.1/haplotypes/main/shapeit/ag1000g.phase1.ar3.1.haplotypes.2L.sample.gz:	 77.5%


In [9]:
!head /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.sample

AB0085-C
AB0087-C
AB0088-C
AB0089-C
AB0090-C
AB0091-C
AB0092-C
AB0094-C
AB0095-C
AB0097-C


In [10]:
!tail /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.sample

AV0032-C
AV0033-C
AV0034-C
AV0035-C
AV0036-C
AV0039-C
AV0041-C
AV0044-C
AV0045-C
AV0047-C


In [11]:
!wc -l /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.sample

765 /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.sample


## haplotype scaffold

In [12]:
# mvncall needs the haps unzipped. Also we will exclude the cross parents
!if [ ! -f /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.haps ]; then gunzip -v {scaffold_file} -c | cut -d' ' -f1-1535 > /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.haps; fi

../ngs.sanger.ac.uk/production/ag1000g/phase1/AR3.1/haplotypes/main/shapeit/ag1000g.phase1.ar3.1.haplotypes.2L.haps.gz:	 98.6%


In [13]:
# check cut has worked
!head -n1 /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.haps

2 . 44688 C T 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [14]:
# check cut has worked
!head -n1 /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.haps | wc

      1    1535    3074


In [16]:
# mvncall needs an unzipped VCF, we'll extract only the region we need
region_vgsc = SeqFeature('2L', 2358158, 2431617)
region_vgsc.region_str

'2L:2358158-2431617'

In [18]:
# extract the VCF
!bcftools view -r {region_vgsc.region_str} --output-file /tmp/vgsc.vcf --output-type v {vcf_file}

In [19]:
%%bash

for numsnps in 50 100 200; do
    echo $numsnps
done

50
100
200


In [20]:
%%bash

# run mvncall, only if output file doesn't exist (it's slow)

mvncall="../dependencies/mvncall_v1.0_x86_64_dynamic/mvncall"
export LD_LIBRARY_PATH=../dependencies

for numsnps in 50 100 200; do
    output_file=../data/phasing_extra_phase1.mvncall.${numsnps}.vcf

    if [ ! -f $output_file ]; then 
        echo running mvncall $numsnps
        $mvncall \
            --sample-file /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.sample \
            --glfs /tmp/vgsc.vcf \
            --scaffold-file /tmp/ag1000g.phase1.ar3.1.haplotypes.2L.haps \
            --list ../data/phasing_extra_phase1.list \
            --numsnps $numsnps \
            --o $output_file > /tmp/mvncall.${numsnps}.log 
    else
        echo skipping mvncall $numsnps
    fi
    
done

running mvncall 50
running mvncall 100
running mvncall 200


In [24]:
!tail /tmp/mvncall.100.log

Analysing variant 3/3

Writing phased genotypes in output files...

Time elapsed : 505.188671

~~~ ΤΕΛΟΣ ~~~ 

Have a nice day!



In [4]:
!cat ../data/phasing_extra_phase1.mvncall.50.vcf

##fileformat=VCFv4.0
##source=MVNcall
##FORMAT=<ID=GT,Number=1,Type=String,Description='Genotype'>
##INFO=<ID=NUMALT,Number=1,Type=Integer,Description='Number of alternative alleles'>
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description='Average posterior probability from MVNcall'>
##FORMAT=<ID=PL,Number=.,Type=Integer,Description='Phred-scaled Genotype Likelihoods'>
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	AB0085-C	AB0087-C	AB0088-C	AB0089-C	AB0090-C	AB0091-C	AB0092-C	AB0094-C	AB0095-C	AB0097-C	AB0098-C	AB0099-C	AB0100-C	AB0101-C	AB0103-C	AB0104-C	AB0109-C	AB0110-C	AB0111-C	AB0112-C	AB0113-C	AB0114-C	AB0117-C	AB0119-C	AB0122-C	AB0123-C	AB0124-C	AB0126-C	AB0127-C	AB0128-C	AB0129-C	AB0130-C	AB0133-C	AB0134-C	AB0135-C	AB0136-C	AB0137-C	AB0138-C	AB0139-C	AB0140-C	AB0142-C	AB0143-C	AB0145-C	AB0146-C	AB0147-C	AB0148-C	AB0151-C	AB0153-C	AB0155-C	AB0157-C	AB0158-C	AB0159-C	AB0160-C	AB0161-C	AB0164-C	AB0166-C	AB0169-C	AB0170-C	AB0171-C	AB0172-C	AB0173-C	AB0174-C	AB0175-C	AB0176-C	AB0177-C	A

In [5]:
!ls -lh ../data/*.mvncall*

-rw-rw-r-- 1 chris chris 894 May 25 16:37 ../data/phasing_extra_phase1.mvncall.100.npz
-rw-rw-r-- 1 chris chris 48K May 25 15:13 ../data/phasing_extra_phase1.mvncall.100.vcf
-rw-rw-r-- 1 chris chris 890 May 25 16:37 ../data/phasing_extra_phase1.mvncall.200.npz
-rw-rw-r-- 1 chris chris 48K May 25 15:47 ../data/phasing_extra_phase1.mvncall.200.vcf
-rw-rw-r-- 1 chris chris 892 May 25 16:37 ../data/phasing_extra_phase1.mvncall.50.npz
-rw-rw-r-- 1 chris chris 48K May 25 15:04 ../data/phasing_extra_phase1.mvncall.50.vcf


## convert to numpy arrays
- so we can interleave these variants back into the genotype array easily

In [30]:
def vcf_to_numpy(numsnps):
    
    # input VCF filename
    vcf_fn = '../data/phasing_extra_phase1.mvncall.{}.vcf'.format(numsnps)
    
    # extract variants
    variants = vcfnp.variants(vcf_fn, cache=False,
                              dtypes={'REF': 'S1', 'ALT': 'S1'},
                              flatten_filter=True)
    # fix the chromosome
    variants['CHROM'] = (b'2L',) * len(variants)

    # extract calldata
    calldata = vcfnp.calldata_2d(vcf_fn, cache=False,
                                 fields=['genotype', 'GT', 'is_phased'])
    # N.B., there is a trailing tab character somewhere in the input VCFs (samples line?) 
    # which means an extra sample gets added when parsing. Hence we will trim off the last
    # field.
    calldata = calldata[:, :-1]
    
    # save output
    output_fn = vcf_fn[:-3] + 'npz'
    np.savez_compressed(output_fn, variants=variants, calldata=calldata)


In [31]:
for numsnps in 50, 100, 200:
    vcf_to_numpy(numsnps)

[vcfnp] 2017-05-25 16:37:02.455790 :: caching is disabled
[vcfnp] 2017-05-25 16:37:02.456622 :: building array
[vcfnp] 2017-05-25 16:37:02.481700 :: caching is disabled
[vcfnp] 2017-05-25 16:37:02.482389 :: building array
[vcfnp] 2017-05-25 16:37:02.537367 :: caching is disabled
[vcfnp] 2017-05-25 16:37:02.538027 :: building array
[vcfnp] 2017-05-25 16:37:02.561956 :: caching is disabled
[vcfnp] 2017-05-25 16:37:02.562624 :: building array
[vcfnp] 2017-05-25 16:37:02.629015 :: caching is disabled
[vcfnp] 2017-05-25 16:37:02.629669 :: building array
[vcfnp] 2017-05-25 16:37:02.649739 :: caching is disabled
[vcfnp] 2017-05-25 16:37:02.650451 :: building array


check parsing...

In [6]:
callset = np.load('../data/phasing_extra_phase1.mvncall.200.npz')
callset

In [7]:
variants = callset['variants']

In [8]:
allel.VariantTable(variants)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER_PASS,num_alleles,is_snp,svlen,AVGPOST,NUMALT
0,b'2L',2391228,b'.',b'G',b'C',0.0,True,3,True,0,0.0,2
1,b'2L',2400071,b'.',b'G',b'A',0.0,True,3,True,0,0.0,2
2,b'2L',2429745,b'.',b'A',b'T',0.0,True,2,True,0,0.0,1


In [9]:
calldata = callset['calldata']

In [10]:
g = allel.GenotypeArray(calldata['genotype'])
g.is_phased = calldata['is_phased']
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764
0,0|0,2|0,0|0,0|0,0|2,0|2,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|2,0|0,0|0,0|0,0|2,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,2|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,2|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|2,0|0,0|0,0|0,0|0,1|0,0|0,0|0,2|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0